# Q1. Getting the embeddings model

In [1]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/Users/vrabinin/Documents/Github/llm-zoomcamp/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
user_question = "I just discovered the course. Can I still join it?"

In [67]:
question_vector = embedding_model.encode(user_question)
print(question_vector[0])

0.07822261


In [14]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()
filtered_documents = [doc for doc in documents if doc.get("course") == 'machine-learning-zoomcamp']
len(filtered_documents)

375

# Q2. Creating the embeddings

In [57]:
embeddings = []
embedded_documents = []
for doc in filtered_documents:
    # Transforming the title into an embedding using the model
    qa_text = f'{doc["question"]} {doc["text"]}'
    doc["qa_embedding"] = embedding_model.encode(qa_text).tolist()
    embeddings.append(doc["qa_embedding"])
    embedded_documents.append(doc)
    len(embeddings)

In [58]:
import numpy as np

X = np.array(embeddings)
X.shape

(375, 768)

# Q3. Search

In [68]:
scores = X.dot(question_vector)
max(scores)

0.6506574349566924

In [69]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=filtered_documents, embeddings=X)
search_engine.search(question_vector, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693',
  'qa_embedding': [0.08062860369682312,
   -0.06663879752159119,
   0.02527310699224472,
   -0.013004404492676258,
   0.07587283849716187,
   -0.05946262925863266,
   -0.021883923560380936,
   0.0029000157956033945,
   0.0007929439889267087,
   -0.005222316365689039,
   -0.03365178033709526,
   -0.02791352942585945,
   0.05811656638979912,
   0.039748288691043854,
   0.054418567568063736,
   -0.03825164958834648,
   0.06305743008

# Q4. Hit-rate for our search engine

In [61]:
import pandas as pd
import requests
from io import StringIO

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

response = requests.get(ground_truth_url, verify=False)

if response.status_code == 200:
    # Use StringIO to convert the text content into a file-like object
    data = StringIO(response.text)
    # Read the data into a DataFrame
    df_ground_truth = pd.read_csv(data)
    df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
    ground_truth = df_ground_truth.to_dict(orient='records')
else:
    print(f"Failed to download data: {response.status_code}")


/Users/vrabinin/Documents/Github/llm-zoomcamp/.venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'github.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/vrabinin/Documents/Github/llm-zoomcamp/.venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'github.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/vrabinin/Documents/Github/llm-zoomcamp/.venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'raw.githubusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/l

In [62]:
def calculate_hit_rate(search_engine, ground_truth):
    num_correct = 0
    for query in ground_truth:
        question = query['question']
        question_vector = embedding_model.encode(question)
        course = query['course']
        document = query['document']
        
        search_results = search_engine.search(question_vector, num_results=5)
        search_documents = [result['id'] for result in search_results]
        
        if document in search_documents:
            num_correct += 1
    
    hit_rate = num_correct / len(ground_truth) * 100
    return hit_rate

hit_rate = calculate_hit_rate(search_engine, ground_truth)
print(f"Hit rate: {hit_rate}%")

Hit rate: 93.98907103825137%


# Q5. Indexing with Elasticsearch

In [63]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '8103cfe6d707', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'olIyrvUQQ2iXypy8GDxUkg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [92]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "id": {"type": "text"},  
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "qa_embedding":{"type":"dense_vector","dims": 768,"index":True,"similarity": "cosine"
        },
        }
    }
}

In [93]:
index_name = "course-questions-hw3"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions-hw3'})

In [94]:
import tqdm

for doc in tqdm.tqdm(embedded_documents):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

100%|██████████| 375/375 [00:00<00:00, 376.81it/s]


In [95]:
es_query = {
    "field" : "qa_embedding",
    "query_vector" :  question_vector,
    "k" : 5,
    "num_candidates" : 10000, 
}

In [96]:
res = es_client.search(index=index_name, knn=es_query,source=["text","section","question","course"])
res["hits"]["hits"][0]["_id"]

'fX07ppAB4hePCQnrU7gH'

# Q6. Hit-rate for Elasticsearch

In [100]:
def calculate_es_hit_rate(es_client, index_name,  ground_truth):
    num_correct = 0
    for query in ground_truth:
        question = query['question']
        question_vector = embedding_model.encode(question)
        course = query['course']
        document = query['document']
        
        es_query = {
            "field" : "qa_embedding",
            "query_vector" :  question_vector,
            "k" : 5,
            "num_candidates" : 10000, 
        }

        search_results = es_client.search(index=index_name, knn=es_query,source=["text","id","section","question","course"])
        search_documents = [result['_source']['id'] for result in search_results["hits"]["hits"]]
        
        if document in search_documents:
            num_correct += 1
    
    hit_rate = num_correct / len(ground_truth) * 100
    return hit_rate

hit_rate = calculate_es_hit_rate(es_client, index_name, ground_truth)
print(f"Hit rate: {hit_rate}%")

Hit rate: 93.98907103825137%
